In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import sys
sys.path.append('..')
from dataset import batcher

In [36]:
MODEL_DIR = '../saved_models/grid_search/'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']
# COUNTRIES = ['ethiopia-2018', 'nigeria-2018', 'malawi-2016']
COUNTRIES = ['ethiopia-2018', 'pollution-2018']

In [31]:
# Get test split
bands = ['BLUE', 'GREEN', 'RED', 'NIR', 'SW_IR1', 'SW_IR2', 'TEMP', 'VIIRS', 'DELTA_TEMP', 'CO']
# bands = ['VIIRS']
def batch(country_year):
    test_batcher = batcher.Batcher(bands, country_year=country_year, bucket=False, shuffle=False, split='custom', repeat=1, batch_size=3000).get_dataset()
    x_true = []
    y_true = []
    for x, y in test_batcher:
        x_true.append(x)
        y_true.append(y)

    y_true = np.squeeze(y_true)
    return x_true, y_true

In [32]:
# Load model
def load_model_from_dir(model_name):
    model_path = MODEL_DIR + model_name + '.h5'
    model = tf.keras.models.load_model(model_path, compile=False)
    
    return model

In [33]:
# Testing purposes only
# grid_search_results = pd.DataFrame(columns=['experiment', 'epoch', 'learning_rate', 'r2'])
# results = {'experiment': "imagery_sample_cnn_regression_0.01_40", 'epoch': 50, 'learning_rate': 1e-2, 'r2': 0.23}
# grid_search_results = grid_search_results.append(results, ignore_index = True)
# display(grid_search_results)

In [37]:
# Hyperparamters tested
lrs = [1e-1, 1e-2, 1e-3, 1e-4]
# epochs = [10, 50, 100, 200]
models = ['sample_cnn', 'deep_sample_cnn']

def run_grid_search(country_year):
    grid_search_results = pd.DataFrame(columns=['experiment', 'model', 'learning_rate', 'r2'])
    x_true, y_true = batch(country_year)

    for m in models:
        for l in lrs:
            experiment_name='imagery_' + m + '_regression' + '_' + str(l)

            # Set hyperparamters
            lr_rate = l
            opt = tf.keras.optimizers.Adam(learning_rate=lr_rate)

            # Get consumption predictions
            trained_model = load_model_from_dir(experiment_name)
            trained_model.compile(
                        loss=LOSS,
                        optimizer=opt,
                        metrics=METRICS,
            )

            y_preds = trained_model.predict_on_batch(x_true)
            y_preds = np.squeeze(y_preds)
            r2 = pearsonr(y_true, y_preds)[0]

            results = {'experiment': experiment_name, 'model': m, 'learning_rate': l, 'r2': r2}
            grid_search_results = grid_search_results.append(results, ignore_index = True)
            
    return grid_search_results


In [38]:
for country_year in COUNTRIES:  
    country_grid_search_results = run_grid_search(country_year)
    print(f'{country_year}:')
    print(country_grid_search_results.sort_values(by=['r2']))

2023-02-23 10:52:13.820828: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/128539193.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-23 10:52:17.323798: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/128539193.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-23 10:52:20.559369: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc

ethiopia-2018:
                                  experiment            model  learning_rate  \
7  imagery_deep_sample_cnn_regression_0.0001  deep_sample_cnn         0.0001   
2        imagery_sample_cnn_regression_0.001       sample_cnn         0.0010   
4     imagery_deep_sample_cnn_regression_0.1  deep_sample_cnn         0.1000   
1         imagery_sample_cnn_regression_0.01       sample_cnn         0.0100   
5    imagery_deep_sample_cnn_regression_0.01  deep_sample_cnn         0.0100   
6   imagery_deep_sample_cnn_regression_0.001  deep_sample_cnn         0.0010   
0          imagery_sample_cnn_regression_0.1       sample_cnn         0.1000   
3       imagery_sample_cnn_regression_0.0001       sample_cnn         0.0001   

         r2  
7 -0.047942  
2 -0.017220  
4 -0.016994  
1  0.021644  
5  0.044883  
6  0.050797  
0  0.058763  
3  0.082905  


2023-02-23 10:52:43.220647: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/128539193.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-23 10:52:46.510942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/128539193.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)
2023-02-23 10:52:49.674150: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc

pollution-2018:
                                  experiment            model  learning_rate  \
2        imagery_sample_cnn_regression_0.001       sample_cnn         0.0010   
7  imagery_deep_sample_cnn_regression_0.0001  deep_sample_cnn         0.0001   
4     imagery_deep_sample_cnn_regression_0.1  deep_sample_cnn         0.1000   
0          imagery_sample_cnn_regression_0.1       sample_cnn         0.1000   
6   imagery_deep_sample_cnn_regression_0.001  deep_sample_cnn         0.0010   
5    imagery_deep_sample_cnn_regression_0.01  deep_sample_cnn         0.0100   
3       imagery_sample_cnn_regression_0.0001       sample_cnn         0.0001   
1         imagery_sample_cnn_regression_0.01       sample_cnn         0.0100   

         r2  
2 -0.081229  
7 -0.071906  
4 -0.058509  
0 -0.011386  
6  0.006295  
5  0.010773  
3  0.028033  
1  0.040370  


/var/folders/yw/3z56n_vx4vq9g05qrxzrr4m40000gn/T/ipykernel_7792/128539193.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid_search_results = grid_search_results.append(results, ignore_index = True)


In [ ]:
"""
Results:

"""